# Web Scraping Gobierno Transparente Chile
##### Exploring wage expenses in the goberment. Public information.

## Attempt 1 Beautiful Soup

In [1]:
import datetime
import os.path
import requests
import urllib.request as ur
from bs4 import BeautifulSoup
import csv

name = 'http://www.gobiernotransparentechile.cl/directorio/entidad/25/351/per_planta/Ao-2015'

# Trying requests
source_code = requests.get(name)
print(source_code)
plain_text = source_code.text
soup = BeautifulSoup(plain_text, "lxml")

# Trying urllib
url_req=ur.Request(name)
url_response=ur.urlopen(url_req)
soup = BeautifulSoup(url_response)
print(soup)


<Response [504]>


HTTPError: HTTP Error 504: GATEWAY_TIMEOUT

Useful Function

In [2]:
def cleanData(df):
   
    columns = ['Apellido paterno', 'Apellido materno', 'Nombres']
    for c in columns:
        df[c] = df[c].str.lower()
        df[c] = df[c].str.replace('á', 'a')
        df[c] = df[c].str.replace('é', 'e')
        df[c] = df[c].str.replace('í', 'i')
        df[c] = df[c].str.replace('ó', 'o')
        df[c] = df[c].str.replace('ú', 'u')
    
    
    df['Remuneración Bruta Mensualizada'] = df['Remuneración Bruta Mensualizada'].replace(to_replace = "\.+", value="", regex=True)

        
    return(df)


        

Beautiful soup did not work. The server time outs when you connect through bsoup.

## Trying Selenium

In [3]:
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import unicodedata

def getDatainPage(url):

    browser = webdriver.Firefox()
    browser.get(url) #This will open a Firefox window on your machine

#    browser.execute_script("window.scrollTo(0,5000)") # Scroll down the page
#    time.sleep(2) # Wait for the scroll
    

    #######
    ### 1 Get table metadata (Year, Department, Kind of contract, ...)
    #######

    table_location_data = browser.find_element_by_class_name("breadcrumb")
    breadcrumb_items = table_location_data.find_elements_by_tag_name("li")
    num_breadcrumbs = len(breadcrumb_items)
    
    department = breadcrumb_items[1].text
    year = breadcrumb_items[num_breadcrumbs-1].text
    type_contract = breadcrumb_items[3].text
    
    #######
    ### 2 Get Data
    #######
    
    data = browser.find_elements_by_tag_name("table")

    for i in data:

        # Headers
        head = i.find_elements_by_tag_name("thead")
        for j in head:
            header_row = j.find_elements_by_tag_name("th")

            # Get length and list of headers
            ncol = len(header_row)

            
            headers = list()
            # Add table location variables
            headers.append("department")
            headers.append("year")
            headers.append("type_contract")
            
            # Now add actual data row values
            for k in header_row:
                headers.append(k.text)

        # Prepare data frame
        df = pd.DataFrame(columns=headers)

        # Actual Data
        table_data = i.find_elements_by_tag_name("tbody")

        for j in table_data:
            data_row = j.find_elements_by_tag_name("tr")

            # Get length and list of data rows
            nrow = len(data_row)        
            extracted_rows = 0
            master_list = list()
            for count, k in enumerate(data_row):
                data_element = k.find_elements_by_tag_name("td")

                # Process only if there is data in the row
                if len(data_element) != 0:

                    actual_record = list()

                    # Add table location variables
                    actual_record.append(department)
                    actual_record.append(year)
                    actual_record.append(type_contract)


                    # Populate list with actual record
                    for l in data_element:
                        actual_record.append(l.text)
                    

                    
                    actual_record_length = len(actual_record)

                    master_list.append(actual_record)
                    extracted_rows += 1

    df = pd.DataFrame(master_list, columns = headers)    
    
    df = cleanData(df)
    
    browser.quit()
    
    return(df)




In [4]:
text_file = open("./data/webpages.csv", "r")

urls = text_file.readlines()

total_urls = len(urls)

print("## Preparing to crawl on",total_urls,"websites")
for count, i in enumerate(urls):

    if count == 0:
        data = getDatainPage(i)
    else:
        data1 = getDatainPage(i)        
        data = pd.concat([data, data1])
    print("Scraped",count+1,"of",total_urls)

    
data.to_csv("./output/scraped_data.csv")    


## Preparing to crawl on 20 websites
'NoneType' object has no attribute 'path'
Scraped 1 of 20
'NoneType' object has no attribute 'path'
Scraped 2 of 20
'NoneType' object has no attribute 'path'
Scraped 3 of 20
'NoneType' object has no attribute 'path'
Scraped 4 of 20


KeyboardInterrupt: 